# Project - Parse Data

### CONFIGURATIONS - SET THIS UPON YOUR RUN TO YOUR PROPER SHORTCUTS TO WRITE DATA OUT APPROPRIATELY

In [ ]:
# 1

### main()
API_KEY = "EWNtZnvIwK9RuVX4WX9Zo19XWGrhget76YEEAxKy"
DATASET_NAME = "s2orc"

### set these to determine which chunks you are going to read in and out
beginning_range_files_process = 130
ending_range_files_process = 135

# 3
### these paths are where the final CSV files will write to
LOCAL_PATH = './drive/MyDrive/DATASCI266/266_full_dataset_automated/'
LOCAL_OUTPUT_CSV_PATH = './drive/MyDrive/DATASCI266/266_full_dataset_parsed/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=9ef4b90fd2e305324ecec067fde0f7deabd6a45d9b320452bbbda2822efce91f
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import gzip
import io
import json
import os
import re
import requests
from tqdm import tqdm
import pandas as pd
import csv
import re

### 1. Read in data from API for specified number of chunks

In [ ]:
# Run this cell to get JSON files

def download_file(url, output_path):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    with open(output_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                file.write(chunk)

### all prints have been commented to speed up processing

def process_batch(batch):
    print(f"Processing batch: {batch['input_metadata_path']}")

    # Ensure directories exist
    os.makedirs(os.path.dirname(batch['input_metadata_path']), exist_ok=True)
    os.makedirs(os.path.dirname(batch['input_pdf_parses_path']), exist_ok=True)

    # Download metadata
    download_file(batch['input_metadata_url'], batch['input_metadata_path'])

    # Download full text
    download_file(batch['input_pdf_parses_url'], batch['input_pdf_parses_path'])
    corpusids_to_keep = set()

    # Filter metadata
    with gzip.open(batch['input_metadata_path'], 'rb') as gz, open(batch['output_metadata_path'], 'wb') as f_out:
        f = io.BufferedReader(gz)
        for line in tqdm(f.readlines(), desc="Filtering metadata"):
            try:
                metadata_dict = json.loads(line)

                # CorpusID
                corpusid = metadata_dict.get('corpusid')
                if not corpusid:
                    # print("Missing corpusid")
                    continue

                # Abstract
                annotations = metadata_dict.get('content', {}).get('annotations', {})
                abstract_info = json.loads(annotations.get('abstract', '[]'))
                if not abstract_info:
                    # print(f"Missing abstract for corpusid: {corpusid}")
                    continue
                abstract_start = abstract_info[0].get('start', 0)
                abstract_end = abstract_info[0].get('end', 0)
                abstract = metadata_dict.get('content', {}).get('text', '')[abstract_start:abstract_end].strip()

                # PaperID
                paperid = metadata_dict.get('content', {}).get('source',{}).get('pdfsha')

                # Write
                f_out.write(line)

            except json.JSONDecodeError as e:
                # print(f"JSONDecodeError: {e}")
                continue
            except TypeError as e:
                # print(f"TypeError: {e}")
                continue
            except Exception as e:
                # print(f"Unexpected error: {e}")
                continue

    # Filter full text
    with gzip.open(batch['input_pdf_parses_path'], 'rb') as gz, open(batch['output_pdf_parses_path'], 'wb') as f_out:
        f = io.BufferedReader(gz)
        for line in tqdm(f.readlines(), desc="Filtering full text"):
            try:
                full_text_dict = json.loads(line)
                corpusid = full_text_dict.get('corpusid')
                f_out.write(line)
            except json.JSONDecodeError as e:
                # print(f"JSONDecodeError: {e}")
                continue
            except TypeError as e:
                # print(f"TypeError: {e}")
                continue
            except Exception as e:
                # print(f"Unexpected error: {e}")
                continue

    # Clean up
    os.remove(batch['input_metadata_path'])
    os.remove(batch['input_pdf_parses_path'])

def main():
    os.makedirs(LOCAL_PATH, exist_ok=True)

    response = requests.get("https://api.semanticscholar.org/datasets/v1/release/latest").json()
    RELEASE_ID = response["release_id"]
    print(f"Latest release ID: {RELEASE_ID}")

    response = requests.get(f"https://api.semanticscholar.org/datasets/v1/release/{RELEASE_ID}/dataset/{DATASET_NAME}/", headers={"x-api-key": API_KEY}).json()

    # CHANGE TO GET NUMBER OF BATCHES (out of 276)
    files_to_process = response["files"][beginning_range_files_process:ending_range_files_process]
    lst_shards = []

    for url in tqdm(files_to_process, desc="Processing batches"):
        match = re.match(r"https://ai2-s2ag.s3.amazonaws.com/staging/(.*)/s2orc/(.*).gz(.*)", url)
        assert match.group(1) == RELEASE_ID
        SHARD_ID = match.group(2)
        print(SHARD_ID)
        lst_shards.append(SHARD_ID)

        batch = {
            'input_metadata_path': os.path.join(LOCAL_PATH, f"{SHARD_ID}_metadata.jsonl.gz"),
            'input_metadata_url': url,
            'input_pdf_parses_path': os.path.join(LOCAL_PATH, f"{SHARD_ID}_pdf_parses.jsonl.gz"),
            'input_pdf_parses_url': url,
            'output_metadata_path': os.path.join(LOCAL_PATH, f"{SHARD_ID}_filtered_metadata.jsonl"),
            'output_pdf_parses_path': os.path.join(LOCAL_PATH, f"{SHARD_ID}_filtered_pdf_parses.jsonl"),
        }
        process_batch(batch)

    print("Downloaded and inspected relevant shards.")
    print(lst_shards)
if __name__ == "__main__":
    main()


Latest release ID: 2024-07-02


Processing batches:   0%|          | 0/5 [00:00<?, ?it/s]

20240705_112719_00076_5gggg_795d90c9-5252-48b9-803b-f479cb99f15b
Processing batch: ./drive/MyDrive/DATASCI266/266_full_dataset_automated/20240705_112719_00076_5gggg_795d90c9-5252-48b9-803b-f479cb99f15b_metadata.jsonl.gz



Filtering metadata: 100%|██████████| 10355/10355 [00:12<00:00, 810.60it/s]

Processing batches:  20%|██        | 1/5 [00:59<03:57, 59.43s/it]

20240705_112719_00076_5gggg_79b76d53-f359-49d6-b915-b80f29c325e8
Processing batch: ./drive/MyDrive/DATASCI266/266_full_dataset_automated/20240705_112719_00076_5gggg_79b76d53-f359-49d6-b915-b80f29c325e8_metadata.jsonl.gz


...

20240705_112719_00076_5gggg_7f6ef21e-3a28-40dc-8d6f-771b1d0cbe7a
Processing batch: ./drive/MyDrive/DATASCI266/266_full_dataset_automated/20240705_112719_00076_5gggg_7f6ef21e-3a28-40dc-8d6f-771b1d0cbe7a_metadata.jsonl.gz



Filtering metadata: 100%|██████████| 53606/53606 [01:11<00:00, 749.16it/s]

Processing batches:  80%|████████  | 4/5 [17:13<04:49, 289.62s/it]

20240705_112719_00076_5gggg_80cff707-f65f-48f8-94d2-45eed2ecb363
Processing batch: ./drive/MyDrive/DATASCI266/266_full_dataset_automated/20240705_112719_00076_5gggg_80cff707-f65f-48f8-94d2-45eed2ecb363_metadata.jsonl.gz



Filtering metadata: 100%|██████████| 53718/53718 [01:13<00:00, 726.30it/s]

Processing batches: 100%|██████████| 5/5 [22:49<00:00, 273.96s/it]

Downloaded and inspected relevant shards.
['20240705_112719_00076_5gggg_795d90c9-5252-48b9-803b-f479cb99f15b', '20240705_112719_00076_5gggg_79b76d53-f359-49d6-b915-b80f29c325e8', '20240705_112719_00076_5gggg_7e23126a-d102-4d27-9fe7-1fca05660a03', '20240705_112719_00076_5gggg_7f6ef21e-3a28-40dc-8d6f-771b1d0cbe7a', '20240705_112719_00076_5gggg_80cff707-f65f-48f8-94d2-45eed2ecb363']


### 2A. Define functions to combine metadata and PDF files

In [ ]:
# JSON to CSV
def extract_info_from_filtered_files(metadata_file, pdf_parses_file):
    data = []
    corpusid_to_metadata = {}

    # Read metadata file to get title and abstract
    with open(metadata_file, 'r') as f:
        for line in f:
            try:
                metadata_dict = json.loads(line)
                corpusid = metadata_dict['corpusid']
                content = metadata_dict['content']

                title_info = json.loads(content['annotations']['title'])[0]
                title_start = title_info['start']
                title_end = title_info['end']
                title = content['text'][title_start:title_end].strip()

                abstract_info = json.loads(content['annotations']['abstract'])[0]
                abstract_start = abstract_info['start']
                abstract_end = abstract_info['end']
                abstract = content['text'][abstract_start:abstract_end].strip()

                paperid = content['source']['pdfsha']

                corpusid_to_metadata[corpusid] = {
                    'title': title,
                    'abstract': abstract,
                    'paperid': paperid
                }
            except Exception as e:
                # print(f"Error processing metadata line: {line}")
                # print(f"Exception: {e}")
                continue

    # Read pdf parses file to get full text
    with open(pdf_parses_file, 'r') as f:
        for line in f:
            try:
                full_text_dict = json.loads(line)
                corpusid = full_text_dict['corpusid']
                if corpusid in corpusid_to_metadata:
                    full_text = full_text_dict['content']['text'].strip()
                    metadata = corpusid_to_metadata[corpusid]
                    data.append({
                        'corpusid': corpusid,
                        'title': metadata['title'],
                        'paperid': metadata['paperid'],
                        'abstract': metadata['abstract'],
                        'full_text': full_text.replace('\n', ' ')  # Replace newlines with spaces
                    })
            except Exception as e:
                # print(f"Error processing full text line: {line}")
                # print(f"Exception: {e}")
                continue

    df = pd.DataFrame(data)

    os.remove(metadata_file)
    os.remove(pdf_parses_file)

    return df

### 2B. Create a dataframe of proper environmental science papers

In [ ]:
path_environmental_id_file = '/content/drive/MyDrive/DATASCI266/envisci_ids.jsonl'

def read_json_lines_file_to_df(file_path):
    data = []
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                try:
                    json_obj = json.loads(line)
                    data.append(json_obj)
                except json.JSONDecodeError:
                    print(f"Error decoding JSON in line: {line}")
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
        return pd.DataFrame()  # Return an empty DataFrame if file not found

    return pd.DataFrame(data)

# Read the JSON Lines file and convert to DataFrame
df_env_ids = read_json_lines_file_to_df(path_environmental_id_file)
df_env_ids['title'] = df_env_ids['title'].str.lower()

df_env_ids_paper_id = df_env_ids[['paperId']]
df_env_ids_paper_id = df_env_ids_paper_id.rename(columns={'paperId':'paperid'})

df_env_ids_title = df_env_ids[['title']]

### 3. Loop through files, combine them into a dataframe, then filter based off list of valid paper IDs

In [ ]:
# Define Constants
regex_pattern = r'full_dataset_automated/(.*?)_filtered'

lst_files_metadata = []
lst_files_pdf = []

# Get list of all written unzipped files

for path, dirnames, filenames in os.walk(LOCAL_PATH):
  for filename in filenames:
    file_path = os.path.join(path, filename)
    if 'metadata' in filename:
      lst_files_metadata.append(file_path)
    elif 'pdf_parses' in filename:
      lst_files_pdf.append(file_path)

# Sort to ensure in same order
lst_files_metadata = sorted(lst_files_metadata)
lst_files_pdf = sorted(lst_files_pdf)
print(f'Metadata Files: {lst_files_metadata}')
print(f'PDF Files: {lst_files_pdf}')

# Extract information from filtered files and write to CSV
for i in range(len(lst_files_metadata)):

  df = pd.DataFrame()
  print('----------')
  print(f'Beginning Loop {i}.')

  # Dynamically create file names and paths based off of unzipped names
  match = re.search(regex_pattern, lst_files_metadata[i])
  paperid_csv_file_name = 'paperid_parsed_dataset' + match.group(1) + '.csv'
  paperid_output_path = LOCAL_OUTPUT_CSV_PATH + paperid_csv_file_name
  title_csv_file_name = 'title_parsed_dataset' + match.group(1) + '.csv'
  title_output_path = LOCAL_OUTPUT_CSV_PATH + title_csv_file_name
  print(f'File Root: {match.group(1)}')

  # convert to dataframe, write to CSV
  df = extract_info_from_filtered_files(lst_files_metadata[i], lst_files_pdf[i])

  df_paperid = pd.merge(df, df_env_ids_paper_id, on='paperid', how = 'inner')
  df_paperid.to_csv(paperid_output_path, index=False, quoting=csv.QUOTE_ALL, lineterminator='\n')
  print(f'Shape of df_paperid = {df_paperid.shape}')

  # df['title'] = df['title'].str.lower()
  # df_title = pd.merge(df, df_env_ids_title, on='title', how = 'inner')
  # df_title.to_csv(title_output_path, index=False, quoting=csv.QUOTE_ALL, lineterminator='\n')
  # print(f'Shape of df_title = {df_title.shape}')
  print(f'Completed Loop {i}')

Metadata Files: ['./drive/MyDrive/DATASCI266/266_full_dataset_automated/20240705_112719_00076_5gggg_795d90c9-5252-48b9-803b-f479cb99f15b_filtered_metadata.jsonl', './drive/MyDrive/DATASCI266/266_full_dataset_automated/20240705_112719_00076_5gggg_79b76d53-f359-49d6-b915-b80f29c325e8_filtered_metadata.jsonl', './drive/MyDrive/DATASCI266/266_full_dataset_automated/20240705_112719_00076_5gggg_7e23126a-d102-4d27-9fe7-1fca05660a03_filtered_metadata.jsonl', './drive/MyDrive/DATASCI266/266_full_dataset_automated/20240705_112719_00076_5gggg_7f6ef21e-3a28-40dc-8d6f-771b1d0cbe7a_filtered_metadata.jsonl', './drive/MyDrive/DATASCI266/266_full_dataset_automated/20240705_112719_00076_5gggg_80cff707-f65f-48f8-94d2-45eed2ecb363_filtered_metadata.jsonl']
PDF Files: ['./drive/MyDrive/DATASCI266/266_full_dataset_automated/20240705_112719_00076_5gggg_795d90c9-5252-48b9-803b-f479cb99f15b_filtered_pdf_parses.jsonl', './drive/MyDrive/DATASCI266/266_full_dataset_automated/20240705_112719_00076_5gggg_79b76d53-f